In [ ]:
# imports

# external modules
import sys
import os
import numpy as np
import pandas as pd
import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
# framework modules
sys.path.append('../')
import plotting.plottools
importlib.reload(plotting.plottools)
from plotting.plottools import plot_histogram
import training.prepare_training_set
importlib.reload(training.prepare_training_set)
from training.prepare_training_set import prepare_training_data_from_files
import training.patternfiltering
importlib.reload(training.patternfiltering)
from training.patternfiltering import contains_any_pattern
import datagen.fake_anomaly
importlib.reload(datagen.fake_anomaly)

In [ ]:
# load the evaluation sets

mes = ([
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1',
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+2',
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+3',
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_-1',
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_-2',
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_-3',
])

data_eras = ([
    'Run2023C-PromptReco-v1',
    'Run2023C-PromptReco-v2',
    'Run2023C-PromptReco-v3',
    'Run2023C-PromptReco-v4',
    'Run2023D-PromptReco-v1',
    'Run2023D-PromptReco-v2',
])

kwargs = ({
    'verbose': True,
    'entries_threshold': 10000,
    'skip_first_lumisections': 5,
    'veto_patterns': [np.zeros((2,2)), np.zeros((2,1)), np.zeros((1,2))]
})

eval_data = {}
for me in mes:
    eval_data[me] = {}
    for era in data_eras:
        files = ['../data/data/ZeroBias-{}-DQMIO-{}_preprocessed.parquet'.format(era, me)]
        (data, runs, lumis) = prepare_training_data_from_files(files, **kwargs)
        eval_data[me][era] = {'data': data, 'runs': runs, 'lumis': lumis}

In [ ]:
# make masks where values are often zero

for me in mes:
    for era in data_eras:
        this_eval_data = eval_data[me][era]['data']
        shape_mask = (np.sum(this_eval_data[:,:,:,0]==0, axis=0)>len(this_eval_data)*0.5)
    
        # temp: also set edges to zero
        shape_mask[0,:] = 1
        shape_mask[-1,:] = 1
        shape_mask[:,0] = 1
        shape_mask[:,-1] = 1
    
        eval_data[me][era]['shape_mask'] = shape_mask

for era in data_eras:
    print('Era {}'.format(era))
    fig,axs = plt.subplots(figsize=(len(mes*6),6), ncols=len(mes))
    if len(mes)==1: axs = [axs]
    for i, me in enumerate(mes):
        plot_histogram(eval_data[me][era]['shape_mask'], fig=fig, ax=axs[i], caxrange=(-0.01,1))
        axs[i].text(0.02, 1.02, 'Shape mask for {}'.format(me.split('_ontrack_')[1]), transform=axs[i].transAxes, fontsize=15)
    plt.show()

In [ ]:
# load keras models

modelbase = '../models/output_20231123/model_20231123'
model_eras = ([
  'Run2023C-v1',
  'Run2023C-v2',
  'Run2023C-v3',
  'Run2023C-v4',
  'Run2023D-v1',
  'Run2023D-v2',
])

models = {}
for me in mes:
    models[me] = {}
    for model_era in model_eras:
        models[me][model_era] = {}
        modelname = '{}_{}_{}.keras'.format(modelbase, model_era, me)
        model = keras.models.load_model(modelname)
        models[me][model_era]['model'] = model

In [ ]:
# load average occupancy or error of training sets

for me in mes:
    for model_era in model_eras:
        avgresponsename = '{}_{}_{}_avgoccupancy.npy'.format(modelbase, model_era, me)
        avgresponse = np.load(avgresponsename)
        avgresponse = np.square(avgresponse)
        models[me][model_era]['avgresponse'] = avgresponse

for model_era in model_eras:
    print('Era {}'.format(era))
    fig,axs = plt.subplots(figsize=(len(mes*6),6), ncols=len(mes))
    if len(mes)==1: axs = [axs]
    for i, me in enumerate(mes):
        plot_histogram(models[me][model_era]['avgresponse'], fig=fig, ax=axs[i])
        axs[i].text(0.02, 1.02, 'Average response for {}'.format(me.split('_ontrack_')[1]), transform=axs[i].transAxes, fontsize=15)
    plt.show()
    
for me in mes:
    for model_era in model_eras:
        avgresponse = models[me][model_era]['avgresponse']
        avgresponse[avgresponse==0] = 1
        avgresponse = np.expand_dims(avgresponse, axis=2)
        models[me][model_era]['avgresponse'] = avgresponse

In [ ]:
# make collections of fake anomalies

nanomalies = 6000
ntimesteps = 1 # note: keep at 1 for now (i.e. no time correction), other options not yet implemented in evaluation below

for me in mes:
    for era in data_eras:
        print('Producing anomalies for {} / {}...'.format(me, era))
        this_eval_data = eval_data[me][era]['data']
        this_shape_mask = eval_data[me][era]['shape_mask']
        random_indices = np.random.choice(len(this_eval_data), size=nanomalies, replace=True)
        random_indices[random_indices<ntimesteps-1] = ntimesteps-1
        anomalies = np.zeros((nanomalies*ntimesteps, this_eval_data.shape[1], this_eval_data.shape[2], 1))
        for newidx,origidx in enumerate(random_indices):
            hist = this_eval_data[origidx-ntimesteps+1:origidx+1,:,:,0]
            (anomalous_hist, paramdict) = datagen.fake_anomaly.dead_rectangle(hist, rectangle_min_pixels=2, shape_mask=~this_shape_mask)
            anomalies[ntimesteps*newidx:ntimesteps*(newidx+1),:,:,0] = anomalous_hist
        eval_data[me][era]['anomalies'] = anomalies

In [ ]:
# define which models to evaluate for which era

model_era_mode = 'previous_and_current'

def get_model_eras(mode='all', data_era=''):
    if mode=='all': return model_eras
    data_era_tag = data_era.replace('-PromptReco-', '-')
    if mode=='previous_and_current':
        eras = [era for era in model_eras if era<=data_era_tag]
        if len(eras)>2: eras = eras[-2:]
        return eras

In [ ]:
# evaluate the models

for me in mes:
    for data_era in data_eras:
        this_eval_data = eval_data[me][data_era]['data']
        this_anomalies = eval_data[me][data_era]['anomalies']
        this_shape_mask = eval_data[me][data_era]['shape_mask']
        eval_data[me][data_era]['errors_time_corrected'] = {}
        eval_data[me][data_era]['errors_anomalies_time_corrected'] = {}
        for model_era in get_model_eras(mode=model_era_mode, data_era=data_era):    
            this_model = models[me][model_era]['model']
            this_avgresponse = models[me][model_era]['avgresponse']
            print('Evaluating model for {} / {} / {}'.format(me, data_era, model_era))
    
            # predictions
            predictions = this_model.predict(this_eval_data)
            predictions[predictions<0] = 0.
            predictions[:,this_shape_mask] = 0.
            predictions_anomalies = this_model.predict(this_anomalies)
            predictions_anomalies[predictions_anomalies<0] = 0.
            predictions_anomalies[:,this_shape_mask] = 0.
    
            # errors
            errors = np.square(this_eval_data - predictions)
            errors[:,this_shape_mask] = 0.
            errors_anomalies = np.square(this_anomalies - predictions_anomalies)
            errors_anomalies[:,this_shape_mask] = 0.

            # space correction
            errors_space_corrected = errors/this_avgresponse
            errors_anomalies_space_corrected = errors_anomalies/this_avgresponse
    
            # extra space correction: divide error by total occupancy of each lumisection
            occupancy_perls = np.square(np.sum(this_eval_data, axis=(1,2))[:,0])
            mean_occupancy_perls = np.mean(occupancy_perls)
            occupancy_perls /= mean_occupancy_perls
            errors_space_corrected = np.divide(errors_space_corrected, occupancy_perls[:,None,None,None])
            occupancy_anomalies_perls = np.square(np.sum(this_anomalies, axis=(1,2))[:,0])
            occupancy_anomalies_perls /= mean_occupancy_perls # use mean from non-anomalous data here for consistency
            errors_anomalies_space_corrected = np.divide(errors_anomalies_space_corrected, occupancy_anomalies_perls[:,None,None,None])
    
            # time correction
            errors_time_corrected = np.zeros(errors_space_corrected.shape)
            for i in range(ntimesteps-1, len(errors_space_corrected)):
                errors_time_corrected[i] = np.prod(errors_space_corrected[i-ntimesteps+1:i+1], axis=0)
            errors_anomalies_time_corrected = np.zeros((nanomalies, this_eval_data.shape[1], this_eval_data.shape[2], this_eval_data.shape[3]))
            for i in range(len(errors_anomalies_time_corrected)):
                errors_anomalies_time_corrected[i:i+1] = np.prod(errors_anomalies_space_corrected[ntimesteps*i:ntimesteps*(i+1)], axis=0)
    
            # remove superfluous anomalies that were only introduced to be able to do the time correction
            #this_anomalies = this_anomalies[::ntimesteps]
            #eval_data[me]['anomalies'] = this_anomalies
            #predictions_anomalies = predictions_anomalies[::ntimesteps]
            #errors_anomalies = errors_anomalies[::ntimesteps]
            #errors_anomalies_space_corrected = errors_anomalies_space_corrected[::ntimesteps]
    
            # add to data structure
            eval_data[me][data_era]['errors_time_corrected'][model_era] = errors_time_corrected
            eval_data[me][data_era]['errors_anomalies_time_corrected'][model_era] = errors_anomalies_time_corrected

In [ ]:
# scan thresholds to make ROC curves

thresholds = [0.25, 0.4, 0.5, 0.6, 0.75, 1, 1.25, 1.5]
patterns = [-np.ones((2,2)), -np.ones((2,1)), -np.ones((1,2))]

for me in mes:
    for data_era in data_eras:
        eval_data[me][data_era]['roc'] = {}
        for model_era in get_model_eras(mode=model_era_mode, data_era=data_era):
            print('Making ROC curve for {} / {} / {}'.format(me, data_era, model_era))
            errors_time_corrected = eval_data[me][data_era]['errors_time_corrected'][model_era]
            errors_anomalies_time_corrected = eval_data[me][data_era]['errors_anomalies_time_corrected'][model_era]
            s_eff = []
            b_eff = []
            for i, threshold in enumerate(thresholds):
                print('  Threshold {} of {}'.format(i+1, len(thresholds)), end='\r')
                errors_scaled = np.where(errors_time_corrected>threshold, -1, errors_time_corrected)
                errors_anomalies_scaled = np.where(errors_anomalies_time_corrected>threshold, -1, errors_anomalies_time_corrected)
                flags = contains_any_pattern(errors_scaled[:,:,:,0], patterns)
                flags_anomalies = contains_any_pattern(errors_anomalies_scaled[:,:,:,0], patterns)
                #print('Threshold: {}'.format(threshold))
                #print('{} out of {} non-anomalous histograms were tagged'.format(sum(flags), len(flags)))
                #print('{} out of {} anomalous histograms were tagged'.format(sum(flags_anomalies), len(flags_anomalies)))
                s_eff.append( sum(flags_anomalies)/len(flags_anomalies) )
                b_eff.append( sum(flags)/len(flags) )
            eval_data[me][data_era]['roc'][model_era] = (s_eff, b_eff)

In [ ]:
# make a summary plot

colors = ({
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1': 'dodgerblue',
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+2': 'darkviolet',
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+3': 'crimson',
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_-1': 'lightskyblue',
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_-2': 'orchid',
    'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_-3': 'orangered',
})

# make the basic figure
fig,axs = plt.subplots(figsize=(3*len(data_eras), 3*len(model_eras)), ncols=len(data_eras), nrows=len(model_eras), squeeze=False)
for j, data_era in enumerate(data_eras):
    for i, model_era in enumerate(model_eras):
        # skip combinations of data eras and model eras that were not evaluated
        if model_era not in get_model_eras(mode=model_era_mode, data_era=data_era):
            axs[i,j].axis('off')
            continue
        # plot the lines for each ME
        for me in mes:
            s_eff, b_eff = eval_data[me][data_era]['roc'][model_era]
            axs[i,j].plot(b_eff, s_eff, label=me.split('_ontrack_')[1], color=colors[me], linewidth=2)
        # plot aesthetics
        axs[i,j].grid()
        axs[i,j].set_ylim((0.5,1))
        axs[i,j].set_xlim((5e-4, 1))
        axs[i,j].set_xscale('log')
        # disable x- and y-axis labels when needed
        if i!=j: axs[i,j].set_xticklabels([])
        if j!=i: axs[i,j].set_yticklabels([])
            
# add the legend
legend = axs[0,0].legend(fontsize=15, loc='upper left', bbox_to_anchor=(4, 1.))

# remove space from between panels
fig.subplots_adjust(wspace=0, hspace=0)

# set axis titles
for i, model_era in enumerate(model_eras):
    axs[i,0].text(-0.2, 0.5, model_era, fontsize=12, va='center', transform=axs[i,0].transAxes, rotation=90)
for j, data_era in enumerate(data_eras):
    axs[-1,j].text(0.5,-0.2, data_era.replace('-PromptReco-','-'), fontsize=12, ha='center', transform=axs[-1,j].transAxes)
fig.text(0.5, 0.05, 'Data era', ha='center', fontsize=15)
fig.text(0.05, 0.5, 'Model era', va='center', fontsize=15, rotation=90)

plt.show()